In [1]:
from veryfi import Client

client_id = 'vrfyAsc0sKUkiuijKFajChAeeibOcI7hfSFKZ43'
client_secret = 'T3LxYm0YAiywWAPr9sVjeTNdO13aCVmlbo9uRw7EbeGU7dT0MArbfes95k39HY0fOvMHfPF8bSEgIG12y463UXKcnVGWDwvCG7ilS6hR3UTmwcL6vo3pZgKZPtn0sktU'
username = 'r.restrepovillegas'
api_key = '19134b45c38ec128d5e1dc6d0c87c706'



# This submits document for processing (takes 3-5 seconds to get response)
veryfi_client = Client(client_id, client_secret, username, api_key)



In [8]:
# Extract information from documents

import os
import json

categories = ['Logistic', 'Internet', 'Services','Telecom', 'Transport']
directory = 'Documents'

for file in os.listdir(directory):
    f = os.path.join(directory, file)
    if os.path.isfile(f):
        base=os.path.basename(f)
        name = os.path.splitext(base)[0]
        print(name)
        response = veryfi_client.process_document(f, categories=categories)
        print(response)
        with open(f'income_data\data_{name}.json', 'w', encoding='utf-8') as f:
            json.dump(response, f, ensure_ascii=False, indent=4)




synth-switch_v5-14
{'account_number': 'O-9589055', 'bill_to': {'address': '25 Thomson Place\n4th Floor', 'name': 'Micro Merchant Systems, Inc.', 'parsed_address': None, 'vat_number': None}, 'cashback': None, 'category': 'Internet', 'created_date': '2024-07-30 14:14:15', 'currency_code': 'USD', 'date': '2024-09-06 00:00:00', 'delivery_date': None, 'discount': None, 'document_reference_number': None, 'document_title': 'Invoice', 'document_type': 'invoice', 'due_date': '2024-05-06', 'duplicate_of': 229109268, 'external_id': None, 'id': 229115548, 'img_file_name': '229115548.pdf', 'img_thumbnail_url': 'https://scdn.veryfi.com/receipts/57f164297e39e848/17d5e3b2-cca1-4bed-94ef-9832aae81eac/thumbnail.jpg?Expires=1722349761&Signature=WE1R~hYma1TJR20tH8tVF~mnQrc0JQJ7JDIppTIPk8TvUE1rq14irNCOcHinoHcN63o9vmK8a5akuqa-qp1RzHTc86GtwwPwr~J7QMzPvaqo34zilcn8o4BDCNt1YEsP0SDdzP5RxflyzXSAIxdktxFdZjAb7UO3CXOhOrKmgWqvS2EtKRDjL7DdokH3DsbG1OofTfV5rXU6uoDs7oh-BqE4Jn4bsv0L9hn~-5x2I9QEqrf3k7LW9mOc98cEw5kU8nePmqW~

In [1]:
import json
from datetime import datetime

def is_valid_invoice_format(data):
    """Check if the JSON data has the expected invoice format."""
    required_keys = {"vendor", "bill_to", "invoice_number", "date", "line_items"}
    if not all(key in data for key in required_keys):
        return False
    
    if not isinstance(data["vendor"], dict) or "name" not in data["vendor"]:
        return False
    
    if not isinstance(data["bill_to"], dict) or "name" not in data["bill_to"]:
        return False
    
    if not isinstance(data["line_items"], list) or len(data["line_items"]) == 0:
        return False
    
    required_item_keys = {"sku", "description", "quantity", "tax_rate", "price", "total"}
    for item in data["line_items"]:
        if not all(key in item for key in required_item_keys):
            return False
    
    return True

def extract_invoice_data(input_json):
    if not is_valid_invoice_format(input_json):
        raise ValueError("Invalid invoice format")

    output = {
        "vendor_name": "",
        "vendor_address": "",
        "bill_to_name": "",
        "invoice_number": "",
        "date": "",
        "line_items": []
    }
    
    output["vendor_name"] = input_json["vendor"].get("name", "")
    output["vendor_address"] = input_json["vendor"].get("address", "")
    output["bill_to_name"] = input_json["bill_to"].get("name", "")
    output["invoice_number"] = input_json.get("invoice_number", "")
    
    date_str = input_json.get("date", "")
    if date_str:
        try:
            date_obj = datetime.strptime(date_str, "%B %d, %Y")
            output["date"] = date_obj.strftime("%Y-%m-%d")
        except ValueError:
            output["date"] = ""
    
    for item in input_json["line_items"]:
        line_item = {
            "sku": item.get("sku", ""),
            "description": item.get("description", ""),
            "quantity": item.get("quantity", ""),
            "tax_rate": item.get("tax_rate", ""),
            "price": item.get("price", ""),
            "total": item.get("total", "")
        }
        output["line_items"].append(line_item)
    
    return output

def process_invoice_file(file_path):
    try:
        with open(file_path, 'r') as file:
            input_json = json.load(file)
        
        output_json = extract_invoice_data(input_json)
        print(json.dumps(output_json, indent=2))
    except json.JSONDecodeError:
        print(f"Error: {file_path} is not a valid JSON file.")
    except ValueError as e:
        print(f"Error processing {file_path}: {str(e)}")
    except Exception as e:
        print(f"Unexpected error processing {file_path}: {str(e)}")

# Example usage
file_paths = [
    'income_data/data_synth-switch_v5-4.json',
    'income_data/invalid_format.json',
    'Documents/synth-switch_v5-4.pdf'
]

for file_path in file_paths:
    print(f"\nProcessing: {file_path}")
    process_invoice_file(file_path)


Processing: income_data/data_synth-switch_v5-4.json
{
  "vendor_name": "switch",
  "vendor_address": "PO Box 674592\nDallas, TX 75267-4592",
  "bill_to_name": "Toni Hackel",
  "invoice_number": "1556267",
  "date": "",
  "line_items": [
    {
      "sku": null,
      "description": "Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\nto HOEpyb (YSPG4VFH) (04/2023)",
      "quantity": 2912.98,
      "tax_rate": null,
      "price": 934.09,
      "total": 2720985.49
    },
    {
      "sku": null,
      "description": "Carrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)",
      "quantity": 3500.87,
      "tax_rate": null,
      "price": 6229.33,
      "total": 21808074.52
    },
    {
      "sku": null,
      "description": "Special Partnership Discount (03/2023)",
      "quantity": 8.0,
      "tax_rate": null,
      "price": -561.0,
      "total": -897

In [10]:
with open('income_data\data_synth-switch_v5-4.json', 'r') as file:
    input_json = json.load(file)

output_json = extract_invoice_data(input_json)
print(json.dumps(output_json, indent=2))

{
  "vendor_name": "switch",
  "vendor_address": "PO Box 674592\nDallas, TX 75267-4592",
  "bill_to_name": "Toni Hackel",
  "invoice_number": "1556267",
  "date": "",
  "line_items": [
    {
      "sku": null,
      "description": "Transport | Switch Fiber Pair (Intra-campus) | Pairs (4419693704) (04/2023|10 Gbps Fiber\nto HOEpyb (YSPG4VFH) (04/2023)",
      "quantity": 2912.98,
      "tax_rate": null,
      "price": 934.09,
      "total": 2720985.49
    },
    {
      "sku": null,
      "description": "Carrier Taxes for Transport | 230 Gbps Wavelength Diverse between Sparks, OR 56789 and\nPlano, NV 98765 (SNpTfT) (NJYM5MQP) (07/2023 Taxes) (07/2023)",
      "quantity": 3500.87,
      "tax_rate": null,
      "price": 6229.33,
      "total": 21808074.52
    },
    {
      "sku": null,
      "description": "Special Partnership Discount (03/2023)",
      "quantity": 8.0,
      "tax_rate": null,
      "price": -561.0,
      "total": -8976.0
    },
    {
      "sku": null,
      "descriptio